In [1]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install functools
!pip install varname
!pip install statsmodels
!pip install sklearn
!pip install seaborn
!pip install xgboost 
!pip install scikit-optimize
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  ERROR: Failed building wheel for functools
  Running setup.py clean for functools
Failed to build functools
    Running setup.py install for functools ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-8zzjn7wx/functools_9b99894b6f2a43daa5e606da2121897a/setup.py'"'"'; __file__='"'"'/tmp/pip-install-8zzjn7wx/functools_9b99894b6f2a43daa5e606da2121897a/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import s

In [2]:
!pip install dask
import dask as dd
import glob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install arch
from arch import arch_model

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 902 kB 8.1 MB/s 
     |████████████████████████████████| 9.8 MB 46.8 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [4]:
import pandas as pd
import numpy as np
from functools import reduce
import matplotlib.pyplot as plt
import os
import sys
import math
from varname import nameof
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels import robust
from statsmodels.tsa.api import VAR
from sklearn.metrics import r2_score
import seaborn as sns

# import a Lasso module for time series analysis
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error

# import Random Forest module
from sklearn.ensemble import RandomForestRegressor

# import XGBoost module
from xgboost import XGBRegressor
from skopt import gp_minimize
from skopt.space import Real, Integer
from functools import partial

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Bidirectional, LSTM, Dropout, SimpleRNN
from tensorflow.keras import backend as K
from tensorflow.keras.backend import clear_session
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression, PLSSVD
from sklearn.preprocessing import scale
from sklearn.model_selection import GridSearchCV
import warnings
#import scipy.optimize as spop
from scipy import stats
from scipy.optimize import minimize
import multiprocessing as multi
warnings.filterwarnings(action='ignore',category=RuntimeWarning)

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
path1 =  '/content/gdrive/MyDrive/garch_month'                   # use your path
all_files = glob.glob(path1 + "/*.csv")

In [6]:
path2='/content/gdrive/MyDrive/filemonth'
files=glob.glob(path2 +"/*.csv")


In [ ]:
##### Implement Garch(1,1) function

def garch_mle(params):
    mu_0=params[0]
    mu_1=params[1]
    mu_2=params[2]
    mu_3=params[3]
    mu_4=params[4]
    mu_5=params[5]
    mu_6=params[6]
    mu_7=params[7]
    mu_8=params[8]
    mu_9=params[9]
    mu_10=params[10]
    omega=params[11]
    alpha=params[12]
    beta=params[13]
    
    
    global R,X
    assert (R.shape[0] == X.shape[0]), "numbers of rows not match!"
    T = X.shape[0]
    dim = X.shape[1]
    # R = R.flatten()
    # X = X.flatten() # flatten arrays

    R1 = np.empty( shape = (0, 1) )
    X1 = np.empty( shape = (0, dim) )
    for t in np.arange(0, T-tau):
        R1 = np.append(R1, R[t+tau].reshape(1, 1), axis = 0)
        X1 = np.append(X1, X[t, :].reshape(1, dim), axis = 0)

    a = np.mat(R1)
    R1=a.T
    X1 = np.mat(X1)
    mean=np.average(R1)
    
    # Long_run volatility
    long_run=omega/(1-alpha-beta)
    
    #calculating realised and conditional volatility
    resid=R1-mu_0-X1[:,0]*mu_1-X1[:,1]*mu_2-X1[:,2]*mu_3-X1[:,3]*mu_4-X1[:,4]*mu_5-X1[:,5]*mu_6-X1[:,6]*mu_7-X1[:,7]*mu_8-X1[:,8]*mu_9-X1[:,9]*mu_10
    
    realised=abs(resid)
    c=np.array(realised)
    conditional=np.zeros(len(R1))
    conditional[0]=long_run
    for t in range(1,len(R1)):
        conditional[t]=omega+alpha*resid[t-1]**2+beta*conditional[t-1]
        
    #calculating log_likelihood
    likelihood=(1/((2*np.pi)**(1/2)*conditional**(1/2)))*np.exp(-c**2/(2*conditional))
    log_likelihood=np.sum(np.log(likelihood))
    
    return -log_likelihood

#### Calculate the ML estimates for one month-horizon multivariate regression models
def multivar_ML(R, X, tau):
    
    assert (R.shape[0] == X.shape[0]), "numbers of rows not match!"
    T = X.shape[0]
    dim = X.shape[1]
    

    R1 = np.empty( shape = (0, 1) )
    X1 = np.empty( shape = (0, dim) )
    for t in np.arange(0, T-tau):
        R1 = np.append(R1, R[t+tau].reshape(1, 1), axis = 0)
        X1 = np.append(X1, X[t, :].reshape(1, dim), axis = 0)

        
    
    # Maximizing log_likelihood
    params=[0.05,0.6,0.6,0.6,0.6,0.6,0.6,0.01,0.5,0.4,0.5,0.1,0.05,0.92]
    res=spop.minimize(garch_mle,params,method="Nelder-Mead",bounds=((-1,1),(-1,1),(-1,1),(-1,1),(-1,1),(-1,1),(-1,1),(-1,1),(-1,1),(-1,1),(-1,1),(0.001,1),(0.001,1),(0.001,1)))
    params=res.x
    mu_0=res.x[0]
    mu=np.mat(res.x[1:11])
     
        
    # compute residuals and residual variance
    epsilon = R1 - mu_0 - (X1 @ mu.T)
    var_res=np.var(epsilon)
    # compute forecast
    forecast_tau = mu_0 + ( mu @ X[T-1, :].reshape(dim, 1) )
    return forecast_tau,var_res

In [136]:
##### Implement principal component regressors

def multivar_lhOLSPC(R, X, num_PCs, tau):
    assert (R.shape[0] == X.shape[0]), "numbers of rows not match!"
    T = X.shape[0]
    dim = X.shape[1]
    # R = R.flatten()
    # X = X.flatten() # flatten arrays
    
    R1 = np.empty( shape = (0, 1) )
    X1 = np.empty( shape = (0, dim) )
    for t in np.arange(0, T):
        if t < T-tau:
            R1 = np.append(R1, R[t+tau].reshape(1, 1), axis = 0)
        else:
            R1 = np.append(R1, np.array([0]).reshape(1, 1), axis = 0)
        X1 = np.append(X1, X[t, :].reshape(1, dim), axis = 0)

    # split the numpy array into train and test data
    if tau == 0:
        X1_train, X1_test = X1[0:(T-1), :], X1[(T-1):, :]
        R1_train, R1_test = R1[0:(T-1), :], R1[(T-1):, :]
    else:
        X1_train, X1_test = X1[0:(T-tau), :], X1[(T-tau):, :]
        R1_train, R1_test = R1[0:(T-tau)], R1[(T-tau):]

    
     # standardizing the features
    X11 = StandardScaler().fit_transform(X1_train)
    X12= StandardScaler().fit_transform(X1_test)
    
    pca = PCA(n_components = num_PCs)
    X_train = pca.fit_transform(X11)
    X_test= pca.transform(X12)
    
    # do time series regressions
    PCs = np.mat(X_train)
    PCs1 = sm.add_constant(PCs)
    PC=np.mat(X_test)
    R1 = np.mat(R1_train)
    ts_res = sm.OLS(R1, PCs1).fit()
    alpha = ts_res.params[0]
    beta = np.mat(ts_res.params[1:])
    
    
    
    # compute residuals
    epsilon = R1 - alpha - (PCs @ beta.T)
    am=arch_model(epsilon, vol="Garch", p=1, o=0, q=1, dist="Normal")
    res=am.fit(update_freq=5)
    forecasts=res.forecast(reindex=False)
    g=forecasts.variance.iloc[-3:]

    
    # forecast for the test data
    forecast_tau=alpha + (beta@PC.T)
    
    return forecast_tau,g.values
    

In [7]:
##### Implement Artificial neural network

def ANNf(R, X, tau, batch_size: int, num_epochs: int):
    assert (R.shape[0] == X.shape[0]), "numbers of rows not match!"
    T = X.shape[0]
    dim = X.shape[1]
    # R = R.flatten()
    # X = X.flatten() # flatten arrays

    R1 = np.empty( shape = (0, 1) )
    X1 = np.empty( shape = (0, dim) )
    for t in np.arange(0, T):
        if t < T-tau:
            R1 = np.append(R1, R[t+tau].reshape(1, 1), axis = 0)
        else:
            R1 = np.append(R1, np.array([0]).reshape(1, 1), axis = 0)
        X1 = np.append(X1, X[t, :].reshape(1, dim), axis = 0)

    # studentize data
    # X1 = StandardScaler().fit_transform(X1)

    # split the numpy array into train and test data
    if tau == 0:
        X1_train, X1_test = X1[0:(T-1), :], X1[(T-1):, :]
        R1_train, R1_test = R1[0:(T-1), :], R1[(T-1):, :]
    else:
        X1_train, X1_test = X1[0:(T-tau), :], X1[(T-tau):, :]
        R1_train, R1_test = R1[0:(T-tau)], R1[(T-tau):]

    # Create the model
    model = Sequential()

    # define the first layer of the ANN
    model.add( Dense( 100, input_shape= (dim, ) ) )
    model.add(Activation('relu'))
    model.add( Dropout(0.5) )

    # define the second layer
    model.add( Dense(50) )
    model.add( Activation('relu') )
    model.add( Dropout(0.5) )

    # define the third layer
    model.add( Dense(10) )
    model.add( Activation('relu') )
    model.add( Dropout(0.5) )

    # define the output layer
    model.add( Dense(1) )
    model.add( Activation('relu') )


    # compile the ANN model
    model.compile(loss='mean_squared_error', optimizer='adam')
    # model.summary()

    # train the model
    model.fit(X1_train, R1_train, batch_size=batch_size, shuffle=False, epochs=num_epochs, verbose=0) # Verbosity mode 0: silent

    # calculate residuals for the train data
    R1_pred = model(X1_train)
    # print('R1_pred = \n', R1_pred)
    residuals = R1_train - R1_pred
    
    var_res=np.var(residuals)


    # forecast for the test data
    forecasts = model(X1_test)
    # print('forecasts = \n', forecasts)
    forecast_tau = float(forecasts[len(forecasts) - 1])
    
    K.clear_session()
    del model # delete the model

    # output results
    if tau == 0:
        return forecast_tau
    else:
        return forecast_tau,var_res
    
    
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)






In [8]:
##### Calculate the mean squared error of XGBoost forecasts for validating samples
def rmsfe_XGBoost(args, R_train, X_train, R_valid, X_valid, seed):
    """   seed               : model seed
    booster : booster to use (gbtree, gblinear or dart; gbtree and dart use tree based models while gblinear uses linear functions)
    n_estimators       : number of boosted trees to fit
    max_depth          : maximum tree depth for base learners
    learning_rate      : boosting learning rate (xgb’s “eta”)
    max_delta_step : maximum step size
    min_child_weight   : minimum sum of instance weight(hessian) needed in a child
    subsample          : subsample ratio of the training instance
    colsample_bytree   : subsample ratio of columns when constructing each tree
    colsample_bylevel  : subsample ratio of columns for each split, in each level
    gamma              :  regularization hyperparameter 
    reg_alpha, reg_lambda: regularization parameters """

    # global models, train_scores, test_scores, curr_model_hyper_params
    curr_model_hyper_params = ['colsample_bylevel', 'colsample_bytree', 'gamma', 'learning_rate', 'max_delta_step', 'max_depth', \
                                                        'min_child_weight', 'n_estimators', 'reg_alpha', 'reg_lambda', 'subsample']
    params = {curr_model_hyper_params[i]: args[i] for i, j in enumerate(curr_model_hyper_params)}
    model = XGBRegressor(booster='gbtree', objective ='reg:squarederror', random_state=42, seed=seed)
    model.set_params(**params)
    model.fit(X_train, R_train) # fit training samples to model
    R_pred = model.predict(X_valid)
    msfe = mean_squared_error(R_valid, R_pred)
    del model
    return msfe

##### Find optimal hyperparameters for XGBoost with cross validation
def minimize_XGBoost(R_train, X_train, R_valid, X_valid, seed = 100, n_calls = 50):
    # defining the space
    space = [
        Real(0.1, 1, name="colsample_bylevel"),
        Real(0.1, 1, name="colsample_bytree"),
        Real(0, 1, name="gamma"),
        Real(0, 1, name="learning_rate"),
        Real(0, 10, name="max_delta_step"),
        Integer(1, 15, name="max_depth"),
        Real(0.1, 500, name="min_child_weight"),
        Integer(10, 100, name="n_estimators"),
        Real(0, 0.5, name="reg_alpha"),
        Real(0, 0.5, name="reg_lambda"),
        Real(0.1, 1, name="subsample"),
    ]

    objective_function = partial(rmsfe_XGBoost, R_train=R_train, X_train=X_train, R_valid=R_valid, X_valid=X_valid, seed=seed)

    # minimize the RMSFE
    res = gp_minimize(objective_function, space, base_estimator=None, n_calls=n_calls, n_random_starts=n_calls-1, random_state=42, n_jobs=1)
    return res.x

##### Implement XGBoost using cross validation
def XGBoostf_CV(R, X, tau: int, seed=1234, n_calls = 150):
    assert (R.shape[0] == X.shape[0]), "numbers of rows not match!"
    assert(tau > 0), "the forecast horizon must be greater than zero!"
    
    T = X.shape[0]
    dim = X.shape[1]
    # R = R.flatten()
    # X = X.flatten() # flatten arrays

    R1 = np.empty( shape = (0, 1) )
    X1 = np.empty( shape = (0, dim) )
    for t in np.arange(0, T):
        if t < T-tau:
            R1 = np.append(R1, R[t+tau].reshape(1, 1), axis = 0)
        else:
            R1 = np.append(R1, np.array([0]).reshape(1, 1), axis = 0)
        X1 = np.append(X1, X[t, :].reshape(1, dim), axis = 0)

    # studentize data
    X1 = StandardScaler().fit_transform(X1)

    # split the numpy array into train, validation, and test data
    sratio = 0.8
    X1_train, X1_valid, X1_test = X1[0:(math.floor(sratio*T)-tau), :].reshape(-1, dim), X1[(math.floor(sratio*T)-tau):(T-tau), :].reshape(-1, dim), \
                                                                                                                                                                                X1[(T-tau):, :].reshape(-1, dim)
    R1_train, R1_valid, R1_test = R1[0:(math.floor(sratio*T)-tau), :].reshape(-1, 1), R1[(math.floor(sratio*T)-tau):(T-tau), :].reshape(-1, 1), \
                                                                                                                                                                                    R1[(T-tau):, :].reshape(-1, 1)

    # find optimal hyperparameters for XGBoost using cross validation
    x = minimize_XGBoost(R1_train, X1_train, R1_valid, X1_valid, seed = seed, n_calls = n_calls)

    # define model with optimal hyperparameters
    model = XGBRegressor(booster='gbtree', objective ='reg:squarederror', seed=seed, colsample_bylevel=x[0], colsample_bytree=x[1], gamma=x[2], \
                                            learning_rate=x[3], max_delta_step=x[4], max_depth=x[5], min_child_weight=x[6], n_estimators=x[7], \
                                            reg_alpha=x[8], reg_lambda=x[9], subsample=x[10])

    # train the model
    X1_train_valid = np.concatenate( (X1_train, X1_valid), axis=0)
    R1_train_valid = np.concatenate( (R1_train, R1_valid), axis=0)
    model.fit(X1_train_valid, R1_train_valid)

    # calculate residuals for the train data
    R1_pred = model.predict(X1_train_valid)
    residuals = R1_train_valid - R1_pred
    resid=residuals[0]
    am=arch_model(resid, vol="Garch",p=1,o=0,q=1,dist="Normal" )
    res=am.fit(update_freq=5)
    forecasts=res.forecast(reindex=False)
    g=forecasts.variance.iloc[-3:]
   
    


    # forecast for the test data
    fore_casts = model.predict(X1_test)
    # print('forecasts = \n', forecasts)
    forecast_tau = float(fore_casts[len(fore_casts) - 1])
    
    del model # delete the model

    # output results
    return forecast_tau, g.values

In [ ]:
from statsmodels.tsa.vector_ar.var_model import forecast
##### Implement Random Forest


# Create a Random Forest model

def create_RF_model(n_estimators=100,max_depth=3,bootstrap=True):

  model=RandomForestRegressor(n_estimators=n_estimators, criterion='squared_error',max_depth=max_depth,bootstrap=bootstrap)
  return model



def randomforestf(R, X, tau: int):
    assert (R.shape[0] == X.shape[0]), "numbers of rows not match!"
    assert(tau > 0), "the forecast horizon must be greater than zero!"
    
    T = X.shape[0]
    dim = X.shape[1]
    # R = R.flatten()
    # X = X.flatten() # flatten arrays

    R1 = np.empty( shape = (0, 1) )
    X1 = np.empty( shape = (0, dim) )
    for t in np.arange(0, T):
        if t < T-tau:
            R1 = np.append(R1, R[t+tau].reshape(1, 1), axis = 0)
        else:
            R1 = np.append(R1, np.array([0]).reshape(1, 1), axis = 0)
        X1 = np.append(X1, X[t, :].reshape(1, dim), axis = 0)

    # studentize data
    X1 = StandardScaler().fit_transform(X1)

    # split the numpy array into train and test data
    X1_train, X1_test = X1[0:(T-tau), :], X1[(T-tau):, :]
    R1_train, R1_test = R1[0:(T-tau)].ravel(), R1[(T-tau):].ravel()

    # Build a Random Forest model
    model= create_RF_model()

    # Define the grid search parameters
    n_estimators=[200,300]
    max_depth=[80,90,100]
    max_features=[3,4]
    min_samples_leaf=[3,4]
    min_samples_split=[8,10]
    bootstrap=[True,False]

    param_grid=dict(n_estimators=n_estimators,max_depth=max_depth,max_features=max_features,min_samples_leaf=min_samples_leaf,bootstrap=bootstrap)

    # Use time-series cross-validation
    tscv=TimeSeriesSplit(n_splits=2, test_size=10)

    # Perform grid search 
    model_cv=GridSearchCV(model,param_grid=param_grid,cv=tscv,refit=True,scoring= "neg_mean_squared_error", n_jobs=-1)
    
    # Cross-validate a model by using the grid search
    model_cv.fit(X1_train,R1_train)
    
    # Random forest model with optimal parameters
    model1=model_cv.best_estimator_
        
    # train the model
    model1.fit(X1_train, R1_train)
       
    # calculate residuals for the train data
    R1_pred = model1.predict(X1_train)
    residuals = R1_train - R1_pred
    am= arch_model(residuals, vol="Garch",p=1,o=0,q=1,dist="Normal")
    res=am.fit(update_freq=5)
    forecasts=res.forecast(reindex=False)
    g=forecasts.variance.iloc[-3:]

    


    # forecast for the test data
    fore_casts = model1.predict( X1_test.reshape(-1, dim) )
    # print('forecasts = \n', forecasts)
    forecast_tau = float(fore_casts[len(fore_casts) - 1])
    
    del model # delete the model


    # output results
    return forecast_tau, g.values




In [ ]:
###### Calculate rolling window Ouf-of-sample R_sq for the Garch(1,1) model

def OoS_R_sq(sstart: int, df: pd.DataFrame, tau, wsize: int, T1: int,ylag: int, use_model)-> tuple:
    """ 'df': a pandas dataframe, 'freq': a frequency of type 'string', e.g., '3M', '1D', '1Y', or '1Q' etc., 'tau': forecast horizon, 'wsize': window size,
            'sstart': sub-sample starting point, 'T1': size of a sub-sample, 'ylag': AR lag of the dependent variable """
    assert (T1 > wsize+tau), "size of a subsample must be much greater than the window size!"
    assert (sstart+T1 <= len(df) ), "end point of the last subsample must be less than or equal to size of the dataframe!"

    df = df.iloc[sstart:(sstart+T1), :].copy()
    # get data for the dependant variable and predictors
    if ylag > 0:
        for i in np.arange(1, ylag+1):
            df[f'ylag{i}'] = df.iloc[:, 1].shift(i)
        df.dropna(inplace = True)
    # print( df.iloc[0, 0] )

    R = np.array(df.values[:, 1], dtype='float64')
    X = np.array(df.values[:, 2:], dtype='float64')
    
    dim = X.shape[1]
    

    rmse = 0
    var = 0
    mae = 0
    #list_err = []
    list_forecast=[]
    list_rol_var_res=[]
    observe_data=[]
    list_rol_var=[]
    list_rolavg=[]
    for s in np.arange(T1-wsize-tau-ylag):
        if use_model=='multivar_ML':
            forecast,var_res= multivar_ML(R[s:(s+wsize+1)].reshape(-1,1), X[s:(s+wsize+1), :].reshape(-1,dim),tau)
            
        
        else:
            print(f'Model {use_model} does not exist!')
            sys.exit()   
        
        
        a=np.mean(R[s:(s+wsize)])
        list_forecast.append(forecast)
        list_rol_var_res.append(var_res)
        observe_data.append(R[(s+wsize+1):(s+wsize+2)])
        r = R[s+wsize+tau] # actual returns
        var1= np.var(R[s:(s+wsize)])
        list_rol_var.append(var1)
        list_rolavg.append(a)
        rmse += pow(r - forecast, 2) / (T1-wsize-tau)
        var += pow(r- a , 2) / (T1-wsize-tau)
        #mae += abs(r - forecast) / (T1-wsize-tau)
    del df # delete this copy of the dataframe
    return  float(1 - rmse/(var + 0.00001)),list_forecast,list_rol_var_res,observe_data,list_rol_var,list_rolavg
   
            
    

In [ ]:
#file=pd.read_csv(all_files[0])

In [ ]:
#b=OoS_R_sq(sstart=0, df=file, tau=1, wsize=85, T1=167, ylag=0, use_model='multivar_ML')

In [ ]:
a1=np.array(b[1])
c1=np.mat(a1)
d1=pd.DataFrame(c1)
d1=d1.T
d1=d1.fillna(0)

a2=np.array(b[2])
c2=np.mat(a2)
d2=pd.DataFrame(c2)
d2=d2.T
d2=d2.fillna(0)

a3=np.array(b[3])
c3=np.mat(a3)
d3=pd.DataFrame(c3)
d3=d3.fillna(0)

d4=pd.DataFrame(b[4])
d4=d4.fillna(0)

q= file['excess_log_return'].ewm(com=0.40).mean()
q=pd.DataFrame(q)
q=q.iloc[86:,]
q=q.values
q=pd.DataFrame(q)

a5=np.array(b[5])
c5=np.mat(a5)
d5=pd.DataFrame(c5)
d5=d5.T
d5=d5.fillna(0)

df=pd.concat([d1,d2,d3,d4,d5,q],axis=1)
df.columns=['return_forecast','rol_var_res','observe_return','rolling_var','rol_avg','ewma']
df.to_csv(r'C:/Users/endou012/emile/sDOC_result/file_63.csv' )


In [138]:
##### Calculate rolling-window OoS R^2 for 'one'-steps ahead forecasts

def OoS_R_sq(sstart: int, df: pd.DataFrame, tau, wsize: int, T1: int, num_PCs:int, ylag: int, use_model)-> tuple:
    """ 'df': a pandas dataframe, 'freq': a frequency of type 'string', e.g., '3M', '1D', '1Y', or '1Q' etc., 'tau': forecast horizon, 'wsize': window size,
            'sstart': sub-sample starting point, 'T1': size of a sub-sample, 'ylag': AR lag of the dependent variable """
    assert (T1 > wsize+tau), "size of a subsample must be much greater than the window size!"
    assert (sstart+T1 <= len(df) ), "end point of the last subsample must be less than or equal to size of the dataframe!"

    df = df.iloc[sstart:(sstart+T1), :].copy()
    # get data for the dependant variable and predictors
    if ylag > 0:
        for i in np.arange(1, ylag+1):
            df[f'ylag{i}'] = df.iloc[:, 1].shift(i)
        df.dropna(inplace = True)
    # print( df.iloc[0, 0] )

    R = np.array(df.values[:, 1], dtype='float64')
    X = np.array(df.values[:, 2:], dtype='float64')
    
    dim = X.shape[1]
    global forecast
    
    rmse = 0
    var = 0
    #mae = 0
    list_forecast=[]
    list_rol_var_pred=[]
    observe_data=[]
    list_rol_var=[]
    list_rolavg=[]
    #list_err = []
    for s in np.arange(T1-wsize-tau-ylag):
    # estimate a long-run regression model and make a 'tau'-steps ahead forecast
        
        if use_model == 'multivar_lhOLSPC':
          r_forecast, var_pred= multivar_lhOLSPC(R[s:(s+wsize+1)].reshape(-1,1),X[s:(s+wsize+1),:].reshape(-1,dim),num_PCs,tau)
                                                                                                            
        
        elif use_model == 'ANNf':
          r_forecast, var_pred= ANNf(R[s:(s+wsize+1)].reshape(-1,1), X[s:(s+wsize+1),:].reshape(-1,dim), tau, batch_size,num_epochs)

                                                                                                                 
        
        elif use_model == 'xgboostf_cv':
          r_forecast, var_pred = XGBoostf_CV(R[s:(s+wsize+1)].reshape(-1,1), X[s:(s+wsize+1),:].reshape(-1,dim),tau,n_calls=50)
                                                                                                                          
        elif use_model == 'randomforestf':
            r_forecast, var_pred= randomforestf(R[s:(s+wsize+1)].reshape(-1, 1), X[s:(s+wsize+1), :].reshape(-1, dim), tau)
        else:
            print(f'Model {use_model} does not exist!')
            sys.exit()

        a=np.mean(R[s:(s+wsize)])
        list_forecast.append(r_forecast)
        list_rol_var_pred.append(var_pred)
        observe_data.append(R[(s+wsize+1):(s+wsize+2)])
        
        r = R[s+wsize+tau] # actual returns
        var1=np.var(R[s:(s+wsize)])
        list_rol_var.append(var1)
        list_rolavg.append(a)

        rmse +=  pow(r - r_forecast, 2) / (T1-wsize-tau)
        var += pow(r - a, 2) / (T1-wsize-tau)
    
    
    del df # delete this copy of the dataframe
    return  float(1 - rmse/(var + 0.00001) ),list_forecast,list_rol_var_pred,observe_data,list_rol_var,list_rolavg
    
    

In [150]:
fil=pd.read_csv(files[1])
fil.shape

(167, 88)

In [151]:

b=OoS_R_sq(sstart=0,df=fil,tau=1,wsize=85,T1=167, num_PCs=10, ylag=1,use_model='multivar_lhOLSPC')

Iteration:      5,   Func. Count:     38,   Neg. LLF: 143.95511033068942
Iteration:     10,   Func. Count:     73,   Neg. LLF: 143.12711799304563
Iteration:     15,   Func. Count:    108,   Neg. LLF: 142.5807012894856
Iteration:     20,   Func. Count:    139,   Neg. LLF: 142.39800869001522
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 142.39800912532576
            Iterations: 20
            Function evaluations: 139
            Gradient evaluations: 20
Iteration:      5,   Func. Count:     37,   Neg. LLF: 145.0501962343518
Iteration:     10,   Func. Count:     69,   Neg. LLF: 144.7731632592836
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 144.77307135120796
            Iterations: 11
            Function evaluations: 75
            Gradient evaluations: 11
Iteration:      5,   Func. Count:     35,   Neg. LLF: 146.88493042139942
Iteration:     10,   Func. Count:     68,   Neg. LLF: 146.79570194729

In [153]:
a1=np.array(b[1])
c1=np.mat(a1)
d1=pd.DataFrame(c1)
d1=d1.T
d1=d1.fillna(0)

a2=np.array(b[2])
c2=np.mat(a2)
d2=pd.DataFrame(c2)
d2=d2.T
d2=d2.fillna(0)

a3=np.array(b[3])
c3=np.mat(a3)
d3=pd.DataFrame(c3)
d3=d3.fillna(0)

d4=pd.DataFrame(b[4])
d4=d4.fillna(0)

a5=np.array(b[5])
c5=np.mat(a5)
d5=pd.DataFrame(c5)
d5=d5.T
d5=d5.fillna(0)

q=fil['excess_log_return'].ewm(com=0.40).mean()
q=pd.DataFrame(q)
q=q.iloc[86:]
q=q.values
q=pd.DataFrame(q)
mer=fil.loc[:,"date"][86:]
mi=mer.values
dmer=pd.DataFrame(mi)



df=pd.concat([dmer,d1,d2,d3,d4,d5,q],axis=1)
df.columns=['date','return_forecast','rol_var_pred','observe_return','rolling_var','rol_avg','ewm']
df.to_csv('/content/gdrive/MyDrive/PCRresult/file_40.csv')



In [152]:
b[0]

-0.0014462918429167804